# Testing RL agents with Stable Baselines 3

Install the conda environment:

```
pip install highway-env
pip install stable-baselines3[extra]
pip install git+https://github.com/DLR-RM/stable-baselines3@feat/gymnasium-support

```

In [1]:
import gymnasium as gym
import highway_env
from stable_baselines3 import DQN

env = gym.make("highway-fast-v0")
model = DQN('MlpPolicy', env,
              policy_kwargs=dict(net_arch=[256, 256]),
              learning_rate=5e-4,
              buffer_size=15000,
              learning_starts=200,
              batch_size=32,
              gamma=0.8,
              train_freq=1,
              gradient_steps=1,
              target_update_interval=50,
              verbose=1,
              tensorboard_log="highway_dqn/")
model.learn(int(2e4))
model.save("highway_dqn/model")

# Load and test saved model
model = DQN.load("highway_dqn/model")
for i in range(1000):
  done = truncated = False
  obs, info = env.reset()
  while not (done or truncated):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = env.step(action)
    env.render()
env.close()

/home/aai/anaconda3/envs/q-learning/lib/python3.8/site-packages/gymnasium/utils/passive_env_checker.py:42: UserWarning: WARN: A Box observation space has an unconventional shape (neither an image, nor a 1D vector). We recommend flattening the observation to have only a 1D vector or use a custom policy to properly process the data. Actual observation shape: (5, 5)
  logger.warn(


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to highway_dqn/DQN_2
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 13.2     |
|    ep_rew_mean      | 9.12     |
|    exploration_rate | 0.975    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 20       |
|    time_elapsed     | 2        |
|    total_timesteps  | 53       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 14.2     |
|    ep_rew_mean      | 10.7     |
|    exploration_rate | 0.946    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 21       |
|    time_elapsed     | 5        |
|    total_timesteps  | 114      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 12.4     |
|

In [5]:
import numpy as np
np.set_printoptions(suppress=True)

In [6]:
obs

array([[ 1.        ,  1.        ,  0.6666667 ,  0.25      ,  0.        ],
       [ 1.        ,  0.00136145, -0.33333334, -0.00138486,  0.        ],
       [ 1.        ,  0.15065317, -0.        ,  0.01258311,  0.        ],
       [ 1.        ,  0.21694683, -0.6666667 ,  0.00030768,  0.        ],
       [ 1.        ,  0.32859212, -0.33333334,  0.00613269,  0.        ]],
      dtype=float32)

In [9]:

for i in range(1000):
  done = truncated = False
  obs, info = env.reset()
  while not (done or truncated):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = env.step(action)
    env.render()
env.close()